In [1]:
import cv2
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from imutils import face_utils
font = cv2.FONT_HERSHEY_SIMPLEX

In [2]:
import os

def get_files(path):
    return os.listdir(path)

cascPath = "/Users/abdulrehman/opt/anaconda3/envs/Face-Detection/lib/python3.6/site-packages/cv2/data/haarcascade_frontalface_default.xml"

def return_bbx(image):
    faceCascade = cv2.CascadeClassifier(cascPath)
    faces = faceCascade.detectMultiScale(image, scaleFactor=1.1, minNeighbors=5, flags=cv2.CASCADE_SCALE_IMAGE)
    return faces

In [3]:
get_files('/Users/abdulrehman/Desktop/SML Project/FacesInTheWild/lfw-deepfunneled')

In [4]:
Dataset_path = '/Users/abdulrehman/Desktop/SML Project/FacesInTheWild/'

Celebs = pd.read_csv(Dataset_path+'lfw_allnames.csv')
Celebs = Celebs[Celebs['images']>50]
# Celebs

In [5]:
# for _,[name,images] in Celebs.iterrows():
#     print(name)
#     print(get_files(Dataset_path+'lfw-deepfunneled/'+name))
#     print('\n\n')

In [6]:
# image = cv2.imread(Dataset_path+'lfw-deepfunneled/Colin_Powell/Colin_Powell_0007.jpg')
# faces = return_bbx(image)
# (x,y,w,h) = faces[0]
# cropped = image[x:x+w, y:y+h]
# plt.imshow(cropped)
# print(cropped.shape)

In [7]:
# r = 64.0 / cropped.shape[1]
# dim = (64, int(cropped.shape[0] * r))
# resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
# plt.imshow(resized)
# print(resized.shape)

In [9]:
X = []
Y = []

for _, [name,__] in Celebs.iterrows():
    celeb_path = Dataset_path+'lfw-deepfunneled/'+name+'/'
    
    images_paths = get_files(celeb_path)
    for image_path in images_paths:
        image = cv2.imread(celeb_path+image_path,1)
        faces = return_bbx(image)
        if len(faces) == 1:
            (x,y,w,h) = faces[0]
            cropped = image[x:x+w, y:y+h]
            dim = (64, 64)
            resized = cv2.resize(cropped, dim, interpolation = cv2.INTER_AREA)
            image = np.array(resized).astype("float32")
            X.append(image)
            Y.append(name)
X_data = np.array(X)
Y_data = np.array(Y)

In [10]:
X_data = np.array(X)
Y_data = np.array(Y)

del X
del Y

print(X_data.shape)
print(Y_data.shape)

(1431, 64, 64, 3)
(1431,)


In [11]:
from collections import Counter

counter = Counter(Y_data)
print(counter)

Counter({'George_W_Bush': 480, 'Colin_Powell': 213, 'Tony_Blair': 138, 'Donald_Rumsfeld': 109, 'Gerhard_Schroeder': 106, 'Ariel_Sharon': 70, 'Hugo_Chavez': 67, 'Jacques_Chirac': 51, 'Jean_Chretien': 51, 'Junichiro_Koizumi': 51, 'Serena_Williams': 48, 'John_Ashcroft': 47})


In [12]:
from imblearn.under_sampling import NearMiss

X_data_reshaped = X_data.reshape(X_data.shape[0], X_data.shape[1] * X_data.shape[2] * X_data.shape[3])
undersample = NearMiss(version=1)
X_resampled, Y_resampled = undersample.fit_resample(X_data_reshaped,Y_data)
X_data = X_resampled.reshape(X_resampled.shape[0], X_data.shape[1], X_data.shape[2], X_data.shape[3])
Y_data = Y_resampled
counter = Counter(Y_data)
print(counter)

del X_data_reshaped
del undersample
del X_resampled
del Y_resampled
del counter

Counter({'Ariel_Sharon': 47, 'Colin_Powell': 47, 'Donald_Rumsfeld': 47, 'George_W_Bush': 47, 'Gerhard_Schroeder': 47, 'Hugo_Chavez': 47, 'Jacques_Chirac': 47, 'Jean_Chretien': 47, 'John_Ashcroft': 47, 'Junichiro_Koizumi': 47, 'Serena_Williams': 47, 'Tony_Blair': 47})


In [13]:
print(X_data.shape)
print(Y_data.shape)

(564, 64, 64, 3)
(564,)


In [14]:
import mahotas

def fd_hu_moments(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image):    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype(int)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    return haralick
 
def fd_histogram(image, mask=None):
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [8, 8, 8], [0, 256, 0, 256, 0, 256])
    # normalize the histogram
    cv2.normalize(hist, hist)
    return hist.flatten()

def get_global_features(image):
    global_feature = np.hstack([fd_histogram(image), fd_haralick(image), fd_hu_moments(image)])
    return global_feature

In [15]:
X_temp = []
for i in range(len(X_data)):
    X_temp.append(get_global_features(X_data[i]))

In [16]:
len(X_temp[0])

532

In [17]:
X_data = np.array(X_temp)
print(X_data.shape)
print(Y_data.shape)

(564, 532)
(564,)


In [18]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder


scaler = MinMaxScaler(feature_range=(0, 1))
X_data = scaler.fit_transform(X_data)
labelencoder = LabelEncoder()
Y_data = labelencoder.fit_transform(Y_data)

In [30]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn import svm

X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, train_size=0.8, random_state = 0)

linear = svm.SVC(kernel='linear', C=2, decision_function_shape='ovo').fit(X_train, y_train)
rbf = svm.SVC(kernel='rbf', gamma=3, C=2, decision_function_shape='ovo').fit(X_train, y_train)
poly = svm.SVC(kernel='poly', degree=3, C=2, decision_function_shape='ovo').fit(X_train, y_train)
sig = svm.SVC(kernel='sigmoid', C=2, decision_function_shape='ovo').fit(X_train, y_train)

In [33]:
accuracy_lin = linear.score(X_train, y_train)
accuracy_poly = poly.score(X_train, y_train)
accuracy_rbf = rbf.score(X_train, y_train)
accuracy_sig = sig.score(X_train, y_train)

print("Training Accuracy Linear Kernel:", accuracy_lin*100)
print("Training Accuracy Polynomial Kernel:", accuracy_poly*100)
print("Training Accuracy Radial Basis Kernel:", accuracy_rbf*100)
print("Training Accuracy Sigmoid Kernel:", accuracy_sig*100)

accuracy_lin = linear.score(X_test, y_test)
accuracy_poly = poly.score(X_test, y_test)
accuracy_rbf = rbf.score(X_test, y_test)
accuracy_sig = sig.score(X_test, y_test)

print("Testing Accuracy Linear Kernel:", accuracy_lin*100)
print("Testing Accuracy Polynomial Kernel:", accuracy_poly*100)
print("Testing Accuracy Radial Basis Kernel:", accuracy_rbf*100)
print("Testing Accuracy Sigmoid Kernel:", accuracy_sig*100)

Training Accuracy Linear Kernel: 56.98447893569845
Training Accuracy Polynomial Kernel: 54.988913525498894
Training Accuracy Radial Basis Kernel: 97.78270509977827
Training Accuracy Sigmoid Kernel: 31.9290465631929
Testing Accuracy Linear Kernel: 28.31858407079646
Testing Accuracy Polynomial Kernel: 27.43362831858407
Testing Accuracy Radial Basis Kernel: 29.20353982300885
Testing Accuracy Sigmoid Kernel: 23.008849557522122
